In [1]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)
import metrics as met
import models as mod
import numpy as np
import preprocessors as pre
import tscv

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
x = np.random.rand(10000000, 10)
y = np.random.rand(10000000)

In [3]:
class StandardScaler(pre.Preprocessor):
    def fit(self, x, y):
        self.mean_ = np.mean(x, axis=0)
        self.std_ = np.std(x, axis=0)

    def transform(self, x, y):
        return ((x - self.mean_) / self.std_, y)

In [4]:
class CustomLinearModel(mod.Model):
    def fit(self, x, y):
        self.coef_ = np.linalg.pinv(x) @ y

    def predict(self, x):
        return x @ self.coef_

In [5]:
# Set up cross-validation
cross_validator = tscv.TimeSeriesCrossValidator(
    x,
    y,
    train_starts=[0, 20000, 40000],
    train_size=100000,
    test_starts=[100000, 120000, 140000],
    test_size=20000,
    metrics=[met.MSE(), met.MAE()]
)

In [6]:
cross_validator.cross_validate(preprocessor=StandardScaler(), model=CustomLinearModel())

,MSE,MAE
0,0.330063,0.497729
1,0.333347,0.499980
2,0.332968,0.499799


In [7]:
cross_validator.cross_validate(preprocessor=StandardScaler(), model=CustomLinearModel(), parallelize="none")

,MSE,MAE
0,0.330063,0.497729
1,0.333347,0.499980
2,0.332968,0.499799


In [8]:
cross_validator.cross_validate(preprocessor=StandardScaler(), model=CustomLinearModel(), parallelize="script", max_workers=8)

,MSE,MAE
0,0.330063,0.497729
1,0.333347,0.499980
2,0.332968,0.499799
